In [1]:
track_progress = True
# If true, the model prints statements as it completes certain long calculations

# Run necessary background code and generate baseline

In [2]:
# Background functions and calculations of adjustment factors
execfile('import_packages.py')
execfile('read_in_data.py')
execfile('amt_model.py')
execfile('ftc_model.py')
execfile('sec199_model.py')
execfile('ccr_model.py')
#execfile('interest_model.py')
execfile('usercode_taxcalc.py')

All packages successfully imported
Data imports complete
Beginning CCR model
Corporate depreciation adjustment calculated
Noncorporate depreciation adjustment calculated


In [3]:
execfile('gen_baseline.py')

Taxable income calculated
Investment matrices constructed
Corporate depreciation calculated
Noncorporate depreciation calculated
Capital paths calculated
Corporate net interest deduction calculated
Noncorporate interest deduction calculated
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
Noncorporate tax rates calculated
Baseline complete


# Specify tax reform

In [4]:
btax_dict1 = {
    2017: {
        'tau_c': 0.357,
        #'tau_nc': 0.0,
        #'tau_amt': 0.2,
        #'depr_3yr_method': 'GDS',
        #'depr_3yr_bonus': 0.5,
        #'depr_5yr_method': 'GDS',
        #'depr_5yr_bonus': 0.5,
        #'depr_7yr_method': 'GDS',
        #'depr_7yr_bonus': 0.5,
        #'depr_10yr_method': 'GDS',
        #'depr_10yr_bonus': 0.5,
        #'depr_15yr_method': 'GDS',
        #'depr_15yr_bonus': 0.5,
        #'depr_20yr_method': 'GDS',
        #'depr_20yr_bonus': 0.5,
        #'depr_25yr_method': 'GDS',
        #'depr_25yr_bonus': 0.0,
        #'depr_275yr_method': 'GDS',
        #'depr_275yr_bonus': 0.0,
        #'depr_39yr_method': 'GDS',
        #'depr_39yr_bonus': 0.0,
        #'depr_land_bonus': 0.0,
        #'inventory_method': 'Mix'
    }
}
btax_dict2 = {
    'undepBasis_corp_hc': {0: 0.0},
    'undepBasis_noncorp_hc': {0: 0.0},
    'amt_repeal': {9e99: False},
    'pymtc_repeal': {9e99: False},
    'ftc_hc': {9e99: 0.0},
    'sec199_hc': {9e99: 0.0},
    'oldIntPaid_corp_hc': {0: 0.0},
    'newIntPaid_corp_hc': {0: 0.0},
    'netIntPaid_corp_hc': {0: 0.0},
    'oldIntPaid_noncorp_hc': {0: 0.0},
    'newIntPaid_noncorp_hc': {0: 0.0}
}
iit_params_ref = {}
execfile('reform_implementation.py')

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
All parameters updated


# Calculate the investment response

In [8]:
# Set all elasticities
#     inv_usercost_c: elasticity of corporate investment w.r.t. the user cost of capital
#     inv_usercost_nc: elasticity of noncorporate investment w.r.t. the user cost of capital
#     inv_eatr: semi-elasticity of investment w.r.t. the effective average tax rate
#     debt_taxshield_c: semi-elasticity of the corporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     debt_taxshield_nc: semi-elasticity of the noncorporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     legalform_ratediff: semi-elasticity of the corporate share of business activity w.r.t. the (Tc - Tnc) tax rate differential


elast_dict = {
    'inv_usercost_c': 0.0,
    'inv_usercost_nc': 0.0,
    'inv_eatr_c': 0.0,
    'inv_eatr_nc': 0.0,
    'mne_share_c': 0.0,
    'mne_share_nc': 0.0,
    'debt_taxshield_c': 0.0,
    'debt_taxshield_nc': 0.0,
    'legalform_ratediff': -1.0
}
execfile('responses.py')

In [6]:
execfile('mini_combined.py')
response_results = inv_response(2017)

In [7]:
response_results

,Asset,deltaIc2014,deltaInc2014,MPKc2014,MPKnc2014,deltaIc2015,deltaInc2015,MPKc2015,MPKnc2015,deltaIc2016,...,MPKc2025,MPKnc2025,deltaIc2026,deltaInc2026,MPKc2026,MPKnc2026,deltaIc2027,deltaInc2027,MPKc2027,MPKnc2027
0,Aerospace products and parts manufacturing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.300354,0.298111,0.0,0.0,0.300354,0.297631,0.0,0.0,0.300354,0.299022
1,Air transportation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.139446,0.126643,0.0,0.0,0.139446,0.126618,0.0,0.0,0.139446,0.126604
2,Aircraft,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.179538,0.173637,0.0,0.0,0.179538,0.173737,0.0,0.0,0.179538,0.173830
3,"All other nonmanufacturing, n.e.c.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.240354,0.238231,0.0,0.0,0.240354,0.237895,0.0,0.0,0.240354,0.239022
4,Amusement and recreation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.121538,0.116298,0.0,0.0,0.121538,0.116359,0.0,0.0,0.121538,0.116407
5,Autos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.448545,0.436229,0.0,0.0,0.448545,0.436186,0.0,0.0,0.448545,0.436182
6,Books,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.237326,0.224333,0.0,0.0,0.237326,0.224283,0.0,0.0,0.237326,0.224252
7,"Chemical manufacturing, ex. pharma and med",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.240354,0.238231,0.0,0.0,0.240354,0.237895,0.0,0.0,0.240354,0.239022
8,Communication,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.124725,0.116436,0.0,0.0,0.124725,0.116442,0.0,0.0,0.124725,0.116447
9,Communications,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.214061,0.205874,0.0,0.0,0.214061,0.205926,0.0,0.0,0.214061,0.205976


In [9]:
legal_response(2017)
print rescale_corp
print rescale_noncorp

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [10]:
print rescale_corp
print rescale_noncorp

[ 1.          1.          1.          0.98442772  0.98443625  0.98449033
  0.98452219  0.98458385  0.98463735  0.98467813  0.98471751  0.9847526
  0.984784    0.98480888]
[ 1.          1.          1.          1.0215174   1.02154883  1.02166838
  1.02174182  1.02187305  1.0219875   1.02206542  1.02214135  1.02221581
  1.02227863  1.02235192]


# Run other responses and recalculate corporate tax revenue

In [11]:
execfile('gen_reform.py')

New investment matrices constructed
New corporate depreciation calculated
New noncorporate depreciation calculated
New capital paths calculated
New earnings calculated
New corporate net interest deduction calculated
New noncorporate interest deduction calculated
New taxable income calculated
Reform corporate tax revenue complete


### Check capital results

In [12]:
capPath_ref_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1027.370558,6192.813634,1105.741653,8525.386334,477.426504,884.575577,2014
1,985.082931,6762.771552,1092.837295,9180.871516,546.326287,905.241147,2015
2,887.419889,7287.566860,915.439816,9843.598272,600.386409,830.188301,2016
3,905.768870,7583.987473,934.368159,10171.241204,626.507756,853.410151,2017
4,943.658335,8015.096696,973.453969,10681.464759,662.121380,808.194361,2018
5,977.892741,8462.516807,1008.769313,11212.587686,698.191016,786.869039,2019
6,1011.648352,8919.671011,1043.590743,11757.067063,734.053025,625.832899,2020
7,1049.819488,9385.154443,1082.967117,12313.109713,769.741271,705.404647,2021
8,1091.369029,9860.097004,1125.828569,12880.899799,805.772796,773.476254,2022
9,1134.752319,10349.107465,1170.581669,13465.625724,842.744263,831.049366,2023


In [13]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1027.370558,6192.813634,1105.741653,8525.386334,477.426504,884.575577,2014
1,985.082931,6762.771552,1092.837295,9180.871516,546.326287,905.241147,2015
2,887.419889,7287.566860,915.439816,9843.598272,600.386409,830.188301,2016
3,920.096877,7703.955635,949.148567,10332.136132,636.418240,866.909916,2017
4,958.577389,8141.813810,988.844086,10850.336631,672.589389,820.971756,2018
5,993.298476,8595.835401,1024.661477,11389.230930,709.190326,799.265383,2019
6,1027.552612,9059.898359,1059.997174,11941.901497,745.593171,635.671702,2020
7,1066.257070,9532.102796,1099.923709,12505.902618,781.793520,716.449543,2021
8,1108.396947,10013.937655,1143.394137,13081.872062,818.344742,785.544298,2022
9,1152.409391,10510.142544,1188.796255,13675.154721,855.857605,843.980733,2023


In [14]:
capPath_ref_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,271.318833,2566.871440,306.337312,6737.647577,159.355609,301.392835,2014
1,287.158825,2686.785985,338.014040,6905.064249,170.218220,328.459158,2015
2,292.681264,2837.223493,306.828509,7157.361498,182.182362,318.973624,2016
3,309.988148,3062.572645,324.971950,7565.731920,197.216674,343.280802,2017
4,322.962484,3236.735092,338.573423,7842.230339,208.431996,329.793859,2018
5,334.699825,3418.483147,350.878108,8132.276560,219.831922,324.001943,2019
6,346.266917,3605.214633,363.004315,8431.387556,231.201186,266.584847,2020
7,359.355781,3796.530678,376.725851,8738.700570,242.558467,297.075021,2021
8,373.599830,3992.456226,391.658410,9052.279025,254.045075,323.502430,2022
9,388.464419,4194.600053,407.241504,9374.469857,265.837200,346.161459,2023


In [15]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,271.318833,2566.871440,306.337312,6737.647577,159.355609,301.392835,2014
1,287.158825,2686.785985,338.014040,6905.064249,170.218220,328.459158,2015
2,292.681264,2837.223493,306.828509,7157.361498,182.182362,318.973624,2016
3,303.458510,2998.062147,318.126691,7406.366186,193.062471,336.049883,2017
4,316.149825,3168.458510,331.431462,7676.804170,204.035274,322.837096,2018
5,327.601237,3345.981157,343.436398,7959.800578,215.169547,317.130245,2019
6,338.898645,3528.498656,355.279885,8251.974623,226.281416,260.912143,2020
7,351.663819,3715.266454,368.662085,8551.649869,237.366536,290.716170,2021
8,365.562034,3906.560722,383.232094,8857.524212,248.579435,316.542453,2022
9,380.077843,4104.042649,398.449550,9172.084016,260.098029,338.688164,2023


### Check combined results

In [16]:
combined_ref

,year,ebitda,taxDep,nid,sec199,taxinc,gbc_adj,tau,taxbc,ftc,amt,pymtc,taxrev
0,2014,2445.993351,884.575577,231.441998,34.306208,1295.669568,0.021643,0.347,421.555664,101.582376,4.457941,3.700228,320.731000
1,2015,2520.645171,905.241147,243.981131,34.855388,1336.567506,0.021643,0.347,434.862110,91.794692,4.529305,3.799722,343.797000
2,2016,2327.856171,830.188301,248.001094,34.816996,1214.849781,0.021643,0.347,395.260349,96.318143,4.524316,3.895522,299.571000
3,2017,2390.549964,853.410151,263.907081,34.407918,1238.824814,0.021643,0.357,415.449051,98.379715,4.345916,3.916140,317.499111
4,2018,2405.788473,808.194361,290.871730,34.473947,1272.248434,0.021643,0.357,426.657909,95.096591,4.354255,3.972608,331.942966
5,2019,2481.147292,786.869039,323.766471,34.801224,1335.710558,0.021643,0.357,447.940401,96.466359,4.395592,4.022942,351.846693
6,2020,2480.796294,625.832899,356.796485,35.301945,1462.864965,0.021643,0.357,490.582571,101.950965,4.458836,4.072006,389.018437
7,2021,2615.370222,705.404647,387.601261,36.035516,1486.328798,0.021643,0.357,498.451340,107.323525,4.551490,4.123058,391.556248
8,2022,2750.945883,773.476254,416.009325,37.026919,1524.433386,0.021643,0.357,511.229996,113.042795,4.676710,4.179542,398.684369
9,2023,2872.888728,831.049366,441.599392,38.376256,1561.863714,0.021643,0.357,523.782532,119.755839,4.847139,4.245000,404.628833


In [17]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.021643,0.347,1295.669568,34.306208,884.575577,231.441998,2445.993351
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.021643,0.347,1336.567506,34.855388,905.241147,243.981131,2520.645171
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.021643,0.347,1214.849781,34.816996,830.188301,248.001094,2327.856171
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.021643,0.347,1258.241448,34.952204,866.909916,268.261581,2428.365149
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.021643,0.347,1292.254685,35.018974,820.971756,295.578104,2443.823519
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.021643,0.347,1356.690115,35.349483,799.265383,328.930336,2520.235317
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.021643,0.347,1485.825688,35.856932,635.671702,362.442904,2519.797226
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.021643,0.347,1509.579935,36.599743,716.449543,393.691242,2656.320464
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.021643,0.347,1548.206251,37.604625,785.544298,422.511919,2793.867094
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.021643,0.347,1586.160018,38.973402,843.980733,448.477548,2917.591701


In [18]:
compare = np.asarray(combined_ref['taxrev'] - combined_base['taxrev'])

print "Total corporate tax revenue change, 2014-2027: " + str(np.round(sum(compare), 3))
print "Total corporate tax revenue change, 2017-2026: " + str(np.round(sum(compare[3:13]), 3))
print "Total corporate tax revenue change, 2018-2027: " + str(np.round(sum(compare[4:]), 3))

Total corporate tax revenue change, 2014-2027: 100.96
Total corporate tax revenue change, 2017-2026: 89.956
Total corporate tax revenue change, 2018-2027: 93.467


In [19]:
for i in compare:
    print i

5.68434188608e-14
-1.13686837722e-13
-1.13686837722e-13
7.49311148508
7.59996633185
7.94769302341
8.66843675302
8.89424771999
9.19436942158
9.50083295272
9.84274201664
10.2158960325
10.5987510339
11.0038862318


## Calculate percent changes in investment income and pass-through income

In [20]:
execfile('multipliers_for_taxcalc.py')

In [21]:
indiv_gfactors

,Investment,year,inc_aftertax_base,inc_aftertax_ref,equity,debt,SchC_pos,SchC_neg,e26270_pos,e26270_neg
0,1105.741653,2014,1416.393849,1416.393849,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1092.837295,2015,1386.540753,1386.540753,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,915.439816,2016,1179.897668,1179.897668,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,949.148567,2017,1213.679328,1182.636016,0.974422,0.983768,1.021502,1.021581,1.021477,1.021534
4,988.844086,2018,1151.313026,1120.852397,0.973543,0.984077,1.021540,1.021586,1.021525,1.021559
5,1024.661477,2019,1138.215655,1107.343112,0.972876,0.984301,1.021663,1.021691,1.021654,1.021674
6,1059.997174,2020,1031.411151,1000.928348,0.970446,0.984421,1.021739,1.021755,1.021733,1.021745
7,1099.923709,2021,1098.173702,1066.471442,0.971132,0.984531,1.021871,1.021881,1.021868,1.021875
8,1143.394137,2022,1163.520433,1130.479393,0.971603,0.984610,1.021986,1.021993,1.021984,1.021989
9,1188.796255,2023,1218.128548,1183.916241,0.971914,0.984663,1.022065,1.022069,1.022063,1.022066


## Pass changes to Tax-Calculator

In [22]:
indiv_rev_impact = distribute_results(iit_params_ref)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [26]:
for x in indiv_rev_impact:
    print x
print sum(indiv_rev_impact[3:])

2.35991903467e-14
7.53214734706e-14
0.0
-3.42752532402
-3.84940666781
-4.12533790447
-5.12709049955
-5.04548513537
-5.05237065838
-5.12222506136
-5.23457594002
-5.36096605876
-5.54323007832
-5.74106489753
-53.6292782256


In [23]:
ModelResults = pd.DataFrame({'year': range(2014,2028), 'IndivTaxRev': indiv_rev_impact})
ModelResults['CorpTaxRev'] = combined_ref['taxrev'] - combined_base['taxrev']
ModelResults['RevenueChange'] = ModelResults['IndivTaxRev'] + ModelResults['CorpTaxRev']
ModelResults

,IndivTaxRev,year,CorpTaxRev,RevenueChange
0,2.359919e-14,2014,5.684342e-14,8.044261e-14
1,7.532147e-14,2015,-1.136868e-13,-3.836536e-14
2,0.000000e+00,2016,-1.136868e-13,-1.136868e-13
3,-9.306648e-01,2017,7.493111e+00,6.562447e+00
4,-1.493852e+00,2018,7.599966e+00,6.106114e+00
5,-1.651356e+00,2019,7.947693e+00,6.296337e+00
6,-2.630810e+00,2020,8.668437e+00,6.037627e+00
7,-2.481289e+00,2021,8.894248e+00,6.412959e+00
8,-2.378804e+00,2022,9.194369e+00,6.815565e+00
9,-2.286767e+00,2023,9.500833e+00,7.214066e+00


In [24]:
print "Total tax revenue change, 2014-2027: " + str(np.round(sum(ModelResults['RevenueChange']), 3))
print "Total tax revenue change, 2017-2026: " + str(np.round(sum(ModelResults['RevenueChange'][3:13]), 3))
print "Total tax revenue change, 2018-2027: " + str(np.round(sum(ModelResults['RevenueChange'][4:]), 3))

Total tax revenue change, 2014-2027: 78.131
Total tax revenue change, 2017-2026: 69.568
Total tax revenue change, 2018-2027: 71.569


In [25]:
for i in range(3,14):
    print ModelResults['RevenueChange'][i]
print '\n'
print sum(ModelResults['RevenueChange'][3:13])

6.56244663922
6.10611433086
6.29633660257
6.03762653414
6.41295866584
6.81556531742
7.21406602661
7.62186931229
8.05489186017
8.44629128402
8.56293978842


69.5681665731
